In [1]:
import pandas as pd
import numpy
from __future__ import division

In [2]:
df1_WS2T = pd.read_csv("/Users/asteriag/Desktop/DATASCIENCEPROJECT_WORKFLOW/TimeStamp_Engineering/engineering_OF_segment1_workflowS2_has_duration.csv",engine='python')

In [3]:
df1_WS2T.head(n=1)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,time_used_str,time_used_second
0,715808|405750|629365,Send for Assessment,Send for Assessment,24/10/2018 9:36:47,14 days 5 hours 50 minutes 45 seconds,1230645.0


In [4]:
df1_WS2T['WRKFLW_TSK_MAP_GROUPED'].nunique()

91

In [5]:
df1_WS2T['WRKFLW_TSK_MAP_DETAILED'].nunique()

184

In [6]:
wd_count = df1_WS2T.groupby('WRKFLW_TSK_MAP_DETAILED').WRKFLW_TSK_MAP_DETAILED.count()
wd_dict = dict()
for i, v in wd_count.items():
    wd_dict[i] = v
sorted_wd_dict = sorted(wd_dict.items(), key = lambda x: x[1], reverse = True)
print(sorted_wd_dict)

[('Send for Assessment', 10814), ('Assessment Decision - Qualified', 7257), ('Assessment Outcome - Successful', 6335), ('Acacdemic - Assessment Decision - Qualified', 2735), ('End My Assessment', 2088), ('Assessment Outcome - Not Successful', 1941), ('Assessment Decision - No Decision', 1773), ('OC - Created - Evidence of course completion', 1573), ('Assessment Decision - Not Qualified', 1112), ('Back to Triage', 611), ('Acacdemic - Assessment Decision - Not Qualified', 583), ('Assessment Outcome - Higher Preference Successful', 558), ('AR - Satisfied - Academic results for prior tertiary studies', 541), ('Put Assessment On-Hold', 528), ('Release Assessment Hold', 432), ('AR - Created - Assessment information (Coursework) (1)', 374), ('Put Triage On-Hold', 332), ('AR - Satisfied - Appointment of authorised representative', 298), ('Release Triage Hold', 277), ('AR - Satisfied - English language proficiency', 271), ('AR - Satisfied - Overseas Student Health Cover details', 241), ('AR - S

In [7]:
list = df1_WS2T.WRKFLW_TSK_MAP_DETAILED.unique()
avg_time_dict = dict()
for key in list :
    df_slides = df1_WS2T[df1_WS2T['WRKFLW_TSK_MAP_DETAILED'] == key]
    sum = df_slides['time_used_second'].sum()
    c = len(df_slides)
    avg = sum//c
    avg_time_dict[key] = avg

sorted_avg_time_dict = sorted(avg_time_dict.items(),key = lambda x: x[1], reverse = True)
print(sorted_avg_time_dict)

[('AR - Reviewed - English language proficiency', 7077756.0), ('AR - Created - Syllabus requirements', 6558623.0), ('AR - Reviewed - Details of authorised representative', 5356897.0), ('AR - Satisfied - Further information (5)', 5328823.0), ('AR - Not Satisfied - Credit for previous study', 4149801.0), ('AR - Satisfied - Syllabus requirements for Electrical / Electrical with Business:;;Is your undergraduate degree a 4 year Engineering degree from a Washington Accord Accredited program and in the same stream as your intended Master of Engineering program?', 3655445.0), ('AR - Created - Overseas Student Health Cover details', 3351209.0), ('OC - Requested - CHESICC transcript authentication for international students', 3198443.0), ('Acacdemic - Assessment Decision - Interview Required', 3171533.0), ('AR - Satisfied - CONDITIONAL Acceptance agreement (International)', 3034653.0), ('OC - Not Satisfied - English language requirements', 3020235.0), ('AR - Satisfied - Academic transcripts for 

In [8]:
numOfWorkflowProcesses = df1_WS2T.groupby('CASE_ID').CASE_ID.count()

count_dic=dict()
for i,v in numOfWorkflowProcesses.items():
    if count_dic.get(v) is None:
        count_dic[v]=[]
        count_dic[v].append(i)
    else:
        count_dic[v].append(i)
#count_dic
count_dic = dict(sorted(count_dic.items()))

In [9]:
len_list=[]
for key in count_dic.keys():
    len_list.append(len(count_dic[key]))

In [10]:
def strfdelta(tdelta, fmt):
    d=dict()
    d["days"],rem1 = divmod(tdelta,60*60*24)
    d["hours"], rem2 = divmod(rem1, 3600)
    d["minutes"], d["seconds"] = divmod(rem2, 60)
    return fmt.format(**d)

In [11]:
list_avgtime_per_key =[]
list_sumtime_row = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_person = 0
    for id in id_list:
        df = df1_WS2T[df1_WS2T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_person+=tsum
    list_avgtime_per_key.append(int((list_sumtime_person/len(count_dic[key]))))
    list_sumtime_row.append(list_sumtime_person)
list_avgtime_per_key_str = []
for i in list_avgtime_per_key:
    list_avgtime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [12]:
list_mediantime_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_per_person = []
    for id in id_list:
        df = df1_WS2T[df1_WS2T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_per_person.append(tsum)
    list_mediantime_per_key.append(numpy.median(list_sumtime_per_person))

list_mediantime_per_key_str = []
for i in list_mediantime_per_key:
    list_mediantime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [13]:
list_percent_reach_end = []
for key in count_dic.keys():
    id_list = count_dic[key]
    count_df_end_step = 0
    count_df_id = 0
    for id in id_list:
        df_id = df1_WS2T[df1_WS2T['CASE_ID']== id]
        df_end_step = df_id[df_id['WRKFLW_TSK_MAP_GROUPED'].isin(['Assessment Outcome - Successful','Assessment Outcome - Not Successful']) ]
        count_df_id += df_id['CASE_ID'].nunique()
        count_df_end_step += df_end_step['CASE_ID'].nunique()
    list_percent_reach_end.append(str(round((count_df_end_step/count_df_id)*100,2))+ "%")

In [14]:
list_step_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = set()
    for id in id_list:
        df = df1_WS2T[df1_WS2T['CASE_ID']== id]
        for index, row in df.iterrows():
            list_step.add(row['WRKFLW_TSK_MAP_GROUPED'])
    list_step_per_key.append(list_step)

In [15]:
from collections import Counter
import collections, functools, operator
list_step_sum_time_per_key = []
list_step_count_per_key = []
list_step_ave_time_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = []
    step_time_list = []
    for id in id_list:
        df = df1_WS2T[df1_WS2T['CASE_ID']== id]
        step_time = dict(zip(df.WRKFLW_TSK_MAP_GROUPED, df.time_used_second))
        step_time_list.append(step_time)
        for index, row in df.iterrows():
            list_step.append(row['WRKFLW_TSK_MAP_GROUPED'])

    step_sum_time = dict(functools.reduce(operator.add,
         map(collections.Counter, step_time_list)))

    dict_list_step=dict(Counter(list_step))

    step_ave_time = {key: strfdelta(step_sum_time[key] // dict_list_step.get(key, 0),"{days} days {hours} hours {minutes} minutes {seconds} seconds")
                        for key in step_sum_time.keys()}

    sort_step_sum_time = sorted(step_sum_time.items(),key = lambda x: x[1], reverse = True)
    list_step_sum_time_per_key.append(sort_step_sum_time)

    sort_dict_list_step = sorted(dict_list_step.items(),key = lambda x: x[1], reverse = True)
    list_step_count_per_key.append(sort_dict_list_step[:6])

    sort_step_ave_time = sorted(step_ave_time.items(),key = lambda x: x[1], reverse = True)
    list_step_ave_time_per_key.append(sort_step_ave_time)

In [16]:
s1_t2=pd.DataFrame({'Number of Workflow processes in this Workflow Segment':count_dic.keys(),'Number of cases':len_list, 'average time for workflow segment':list_avgtime_per_key_str,'median time for workflow segment':list_mediantime_per_key_str,'percentage of cases reaching to end of workflow segment':list_percent_reach_end})
s1_t2['Num of Occurrences per step(descending)'] = list_step_count_per_key
s1_t2['Average time spends per step(descending)'] = list_step_ave_time_per_key
s1_t2['average time for workflow segment(seconds)'] = list_avgtime_per_key
s1_t2['sum time for workflow segment(seconds)'] = list_sumtime_row
s1_t2.head(n=10)


,Number of Workflow processes in this Workflow Segment,Number of cases,average time for workflow segment,median time for workflow segment,percentage of cases reaching to end of workflow segment,Num of Occurrences per step(descending),Average time spends per step(descending),average time for workflow segment(seconds),sum time for workflow segment(seconds)
0,1,835,0 days 0 hours 0 minutes 0 seconds,0.0 days 0.0 hours 0.0 minutes 0.0 seconds,0.0%,"[(Send for Assessment, 835)]",[],0,0.000000e+00
1,2,854,26 days 15 hours 4 minutes 24 seconds,15.0 days 18.0 hours 21.0 minutes 6.0 seconds,62.65%,"[(Send for Assessment, 855), (Assessment Outco...","[(Send for Assessment, 26.0 days 14.0 hours 19...",2300664,1.964768e+09
2,3,2807,30 days 13 hours 9 minutes 23 seconds,18.0 days 22.0 hours 25.0 minutes 54.0 seconds,91.09%,"[(Send for Assessment, 2809), (Assessment Outc...","[(AR - Satisfied - specialisation, 94.0 days 2...",2639363,7.408693e+09
3,4,2170,39 days 22 hours 5 minutes 52 seconds,27.0 days 9.0 hours 52.0 minutes 17.5 seconds,92.07%,"[(Send for Assessment, 2180), (Assessment Deci...","[(AR - Created - further information, 8.0 days...",3449152,7.484660e+09
4,5,1568,45 days 20 hours 41 minutes 49 seconds,30.0 days 2.0 hours 15.0 minutes 11.5 seconds,91.33%,"[(Send for Assessment, 1594), (Assessment Deci...","[(AR - Reviewed - further information, 9.0 day...",3962509,6.213215e+09
5,6,779,53 days 4 hours 43 minutes 27 seconds,36.0 days 22.0 hours 52.0 minutes 12.0 seconds,90.12%,"[(Assessment Decision - Qualified, 865), (Send...","[(AR - Satisfied - work experience, 8.0 days 7...",4596207,3.580446e+09
6,7,431,67 days 8 hours 50 minutes 12 seconds,45.0 days 2.0 hours 35.0 minutes 44.0 seconds,80.74%,"[(Assessment Decision - Qualified, 509), (Send...","[(Assessment Outcome - Ineligible / Withdrawn,...",5820612,2.508684e+09
7,8,231,62 days 20 hours 36 minutes 2 seconds,41.0 days 21.0 hours 58.0 minutes 37.0 seconds,87.01%,"[(Send for Assessment, 298), (Assessment Decis...","[(Send for Assessment, 9.0 days 14.0 hours 59....",5430962,1.254552e+09
8,9,158,60 days 2 hours 59 minutes 49 seconds,42.0 days 10.0 hours 59.0 minutes 46.5 seconds,87.34%,"[(Assessment Decision - Qualified, 223), (Send...","[(Release Assessment Hold, 8.0 days 4.0 hours ...",5194789,8.207768e+08
9,10,125,85 days 19 hours 49 minutes 3 seconds,48.0 days 6.0 hours 12.0 minutes 16.0 seconds,86.4%,"[(Assessment Decision - Qualified, 187), (Send...",[(Acacdemic - Assessment Decision - No Decisio...,7415343,9.269179e+08


In [17]:
s1_t2.to_csv('engineering_OF_segment1_workflowS2_table.csv', index=False)









